This code is responsible for generating the sustainability costs for each provider, model and prompt. We use the EcoLogits lib, available at https://ecologits.ai/latest/

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Read the API key
API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

API_KEY_AZURE = os.getenv("AZURE_API_KEY")
ENDPOINT_AZURE = os.getenv("AZURE_API_ENDPOINT")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
import os


inputUser = """
Here is the entity extracted from the database that you should seek information about:

iata: "GRU"
country: "Brazil"
city: "Guarulhos"

context: [NO_CONTEXT_ADDED].
"""

def getPrompts(promptsPath="../prompts/"):
    promptsFiles = os.listdir(promptsPath)
    prompts = []

    for filename in promptsFiles:
        if filename.endswith(".txt"):
            file_path = os.path.join(promptsPath, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                content.replace("[PROPERTIES]", "name, icao, runways")
                prompts.append(content)

    promptsFiles = [x.replace('.txt','') for x in promptsFiles]
    return promptsFiles, prompts

files, prompts = getPrompts()

In [21]:
from ecologits import EcoLogits
from openai import OpenAI
import numpy

# Initialize EcoLogits
EcoLogits.init(providers=["openai"])

models = ['gpt-5','gpt-5-mini']

resultsOpenAi = {}

client = OpenAI(api_key=OPENAI_KEY)
for model in models:
    resultsOpenAi[model] = {}
    for index, prompt in enumerate(prompts):
        valuesEnergy = []
        valuesGwp = []
        valuesWcf = []
        for x in range(5):
            response = client.chat.completions.create( 
            model=model,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": inputUser},
            ]
            )

            valuesEnergy.append(response.impacts.energy.value.mean)
            valuesGwp.append(response.impacts.gwp.value.mean)
            valuesWcf.append(response.impacts.wcf.value.mean)


        resultsOpenAi[model][files[index]] = [numpy.average(valuesEnergy),numpy.average(valuesWcf),numpy.average(valuesGwp)]

print(resultsOpenAi)

{'gpt-5': {'compact': [np.float64(0.004177888916284763), np.float64(0.01807989784300064), np.float64(0.0016779481194958614)], 'few5': [np.float64(0.0021029632669997073), np.float64(0.009100615597206574), np.float64(0.0008522365715255695)], 'few3': [np.float64(0.001679605893550605), np.float64(0.007268528096458114), np.float64(0.000680875583460828)], 'few1': [np.float64(0.0018795799859624481), np.float64(0.008133919980852214), np.float64(0.0007599245987778092)], 'zero': [np.float64(0.001567665818194937), np.float64(0.006784105181554953), np.float64(0.0006409026669189125)], '5cot': [np.float64(0.002310335390213688), np.float64(0.009998022607857539), np.float64(0.0009425623248018701)], 'zerocot': [np.float64(0.0010962987213759144), np.float64(0.004744254642728697), np.float64(0.00045403022668282194)], '3cot': [np.float64(0.0018860992345758492), np.float64(0.008162132159611677), np.float64(0.0007734119022386425)], '1cot': [np.float64(0.0019219569659500192), np.float64(0.008317307209288026)

In [22]:
# Antropic

from ecologits import EcoLogits
from anthropic import Anthropic

# Initialize EcoLogits
EcoLogits.init(providers=["anthropic"])

client = Anthropic(api_key=ANTHROPIC_API_KEY)

resultsClaude = {}

models = ['claude-sonnet-4-5', 'claude-haiku-4-5']

for model in models:
    resultsClaude[model] = {}
    for index, prompt in enumerate(prompts):
        valuesEnergy = []
        valuesGwp = []
        valuesWcf = []
        for x in range(5):
            response = client.messages.create( 
            max_tokens=500,
            messages=[
                {"role": "user", "content": inputUser}
                ],
            system=prompt,
            model=model,
            )
        
            valuesEnergy.append(response.impacts.energy.value.mean)
            valuesGwp.append(response.impacts.gwp.value.mean)
            valuesWcf.append(response.impacts.wcf.value.mean)

        resultsClaude[model][files[index]] = [numpy.average(valuesEnergy),numpy.average(valuesWcf),numpy.average(valuesGwp)]

print(resultsClaude)

{'claude-sonnet-4-5': {'compact': [np.float64(0.00043639755418444875), np.float64(0.0018251041733479373), np.float64(0.00018399941247499094)], 'few5': [np.float64(8.142824189368358e-05), np.float64(0.0003416026735554822), np.float64(3.74483056745914e-05)], 'few3': [np.float64(7.629741904484775e-05), np.float64(0.00032013685287586244), np.float64(3.510727769618575e-05)], 'few1': [np.float64(0.0001241154106925579), np.float64(0.0005172573995143604), np.float64(5.5802596893834576e-05)], 'zero': [np.float64(0.00034582536047389554), np.float64(0.0014455881170979121), np.float64(0.00014683736629381603)], '5cot': [np.float64(0.00022989944341972462), np.float64(0.0009618238038854272), np.float64(9.787112180701593e-05)], 'zerocot': [np.float64(0.0006288064490549422), np.float64(0.00263289592121455), np.float64(0.00026073737423909407)], '3cot': [np.float64(8.003641528420017e-05), np.float64(0.0003355454398068496), np.float64(3.664036257068876e-05)], '1cot': [np.float64(0.000537554915086381), np.

In [ ]:
# Gemini

models = ['gemini-2.5-pro','gemini-2.5-flash']

from ecologits import EcoLogits
from google import genai
from google.genai import types

# Initialize EcoLogits
EcoLogits.init(providers=["google_genai"])

resultsGemini = {}

client = genai.Client(api_key=API_KEY)
for model in models:
    resultsGemini[model] = {}
    for index, prompt in enumerate(prompts):
        valuesEnergy = []
        valuesGwp = []
        valuesWcf = []
        for x in range(5):

            response = client.models.generate_content(
            model=model,
                config=types.GenerateContentConfig(
            system_instruction=prompt,tools=[]),
            contents=inputUser,
            )
            valuesEnergy.append(response.impacts.energy.value.mean)
            valuesGwp.append(response.impacts.gwp.value.mean)
            valuesWcf.append(response.impacts.wcf.value.mean)
        
        resultsGemini[model][files[index]] = [numpy.average(valuesEnergy),numpy.average(valuesWcf),numpy.average(valuesGwp)]

print(resultsGemini)

{'gemini-2.5-pro': {'compact': [np.float64(0.027824443255947018), np.float64(0.12278896201961835), np.float64(0.01078678034219228)], 'few5': [np.float64(0.028831448012147654), np.float64(0.12723286293167946), np.float64(0.011149360277679038)], 'few3': [np.float64(0.02739485520243724), np.float64(0.12089319466494827), np.float64(0.01060233928809603)], 'few1': [np.float64(0.030039771543695925), np.float64(0.13256518138484313), np.float64(0.011627932283269443)], 'zero': [np.float64(0.02825856142943022), np.float64(0.12470472074389982), np.float64(0.010945630365076746)], '5cot': [np.float64(0.029496036596334957), np.float64(0.1301656850432236), np.float64(0.011409259537009692)], 'zerocot': [np.float64(0.02639256051248735), np.float64(0.11647007922344103), np.float64(0.010216564803059911)], '3cot': [np.float64(0.036282338481568234), np.float64(0.16011356061343732), np.float64(0.014040355534732018)], '1cot': [np.float64(0.026414135520983695), np.float64(0.1165652894986103), np.float64(0.0102

In [25]:
import pandas as pd
import numpy as np 
data = {'gemini-2.5-pro': {'compact': [np.float64(0.027824443255947018), np.float64(0.12278896201961835), np.float64(0.01078678034219228)], 'few5': [np.float64(0.028831448012147654), np.float64(0.12723286293167946), np.float64(0.011149360277679038)], 'few3': [np.float64(0.02739485520243724), np.float64(0.12089319466494827), np.float64(0.01060233928809603)], 'few1': [np.float64(0.030039771543695925), np.float64(0.13256518138484313), np.float64(0.011627932283269443)], 'zero': [np.float64(0.02825856142943022), np.float64(0.12470472074389982), np.float64(0.010945630365076746)], '5cot': [np.float64(0.029496036596334957), np.float64(0.1301656850432236), np.float64(0.011409259537009692)], 'zerocot': [np.float64(0.02639256051248735), np.float64(0.11647007922344103), np.float64(0.010216564803059911)], '3cot': [np.float64(0.036282338481568234), np.float64(0.16011356061343732), np.float64(0.014040355534732018)], '1cot': [np.float64(0.026414135520983695), np.float64(0.1165652894986103), np.float64(0.010219314373311233)]}, 'gemini-2.5-flash': {'compact': [np.float64(0.001978620696224292), np.float64(0.008731631367610142), np.float64(0.0007728685430937055)], 'few5': [np.float64(0.0016179214963419695), np.float64(0.0071398697662206495), np.float64(0.0006293030305468316)], 'few3': [np.float64(0.0021865268264018166), np.float64(0.009649118833116126), np.float64(0.0008514907735119369)], 'few1': [np.float64(0.0016415855589572898), np.float64(0.007244299014237371), np.float64(0.0006390147457298779)], 'zero': [np.float64(0.002104011928170524), np.float64(0.009284981494885314), np.float64(0.0008206803061710163)], '5cot': [np.float64(0.002243686026974385), np.float64(0.009901361756491662), np.float64(0.0008721760348755547)], 'zerocot': [np.float64(0.002707633620984401), np.float64(0.011948757385434331), np.float64(0.0010564766128775837)], '3cot': [np.float64(0.002652208683102205), np.float64(0.011704167744234514), np.float64(0.0010331343549585467)], '1cot': [np.float64(0.002218102284408761), np.float64(0.009788460981970731), np.float64(0.0008644319637983701)]}, 'claude-sonnet-4-5': {'compact': [np.float64(0.00043639755418444875), np.float64(0.0018251041733479373), np.float64(0.00018399941247499094)], 'few5': [np.float64(8.142824189368358e-05), np.float64(0.0003416026735554822), np.float64(3.74483056745914e-05)], 'few3': [np.float64(7.629741904484775e-05), np.float64(0.00032013685287586244), np.float64(3.510727769618575e-05)], 'few1': [np.float64(0.0001241154106925579), np.float64(0.0005172573995143604), np.float64(5.5802596893834576e-05)], 'zero': [np.float64(0.00034582536047389554), np.float64(0.0014455881170979121), np.float64(0.00014683736629381603)], '5cot': [np.float64(0.00022989944341972462), np.float64(0.0009618238038854272), np.float64(9.787112180701593e-05)], 'zerocot': [np.float64(0.0006288064490549422), np.float64(0.00263289592121455), np.float64(0.00026073737423909407)], '3cot': [np.float64(8.003641528420017e-05), np.float64(0.0003355454398068496), np.float64(3.664036257068876e-05)], '1cot': [np.float64(0.000537554915086381), np.float64(0.0022499288403577135), np.float64(0.00022415264510751575)]}, 'claude-haiku-4-5': {'compact': [np.float64(1.406408725363135e-05), np.float64(6.0278996034466996e-05), np.float64(5.8345814037265345e-06)], 'few5': [np.float64(3.5740881296604645e-06), np.float64(1.5351014746063666e-05), np.float64(1.649119539520178e-06)], 'few3': [np.float64(3.6906722830952747e-06), np.float64(1.5847474360469012e-05), np.float64(1.6945926518988438e-06)], 'few1': [np.float64(5.207947857766727e-06), np.float64(2.2293475815643537e-05), np.float64(2.257121746321098e-06)], 'zero': [np.float64(1.098723815057072e-05), np.float64(4.7134499042986825e-05), np.float64(4.8044900625925115e-06)], '5cot': [np.float64(1.0395845474435754e-05), np.float64(4.4544110205032084e-05), np.float64(4.394442486947949e-06)], 'zerocot': [np.float64(3.154831788963059e-05), np.float64(0.00013529328796890678), np.float64(1.2823361912751315e-05)], '3cot': [np.float64(1.3877316287663858e-05), np.float64(5.9465951684614314e-05), np.float64(5.800491658816543e-06)], '1cot': [np.float64(2.4883761543930505e-05), np.float64(0.0001066610885105763), np.float64(1.0293007876467048e-05)]}, 'gpt-5': {'compact': [np.float64(0.004177888916284763), np.float64(0.01807989784300064), np.float64(0.0016779481194958614)], 'few5': [np.float64(0.0021029632669997073), np.float64(0.009100615597206574), np.float64(0.0008522365715255695)], 'few3': [np.float64(0.001679605893550605), np.float64(0.007268528096458114), np.float64(0.000680875583460828)], 'few1': [np.float64(0.0018795799859624481), np.float64(0.008133919980852214), np.float64(0.0007599245987778092)], 'zero': [np.float64(0.001567665818194937), np.float64(0.006784105181554953), np.float64(0.0006409026669189125)], '5cot': [np.float64(0.002310335390213688), np.float64(0.009998022607857539), np.float64(0.0009425623248018701)], 'zerocot': [np.float64(0.0010962987213759144), np.float64(0.004744254642728697), np.float64(0.00045403022668282194)], '3cot': [np.float64(0.0018860992345758492), np.float64(0.008162132159611677), np.float64(0.0007734119022386425)], '1cot': [np.float64(0.0019219569659500192), np.float64(0.008317307209288026), np.float64(0.0007945532917193245)]}, 'gpt-5-mini': {'compact': [np.float64(0.00042277100504773855), np.float64(0.001829549979764189), np.float64(0.00017011133645376326)], 'few5': [np.float64(0.00022858209982801458), np.float64(0.0009891936086477294), np.float64(9.244890672544819e-05)], 'few3': [np.float64(0.0002558920710157229), np.float64(0.0011073780551619612), np.float64(0.00010380024115720684)], 'few1': [np.float64(0.0002944206359339112), np.float64(0.0012741111904167194), np.float64(0.00011877193250485795)], 'zero': [np.float64(0.00014230480333931559), np.float64(0.000615826882546955), np.float64(5.799704637367925e-05)], '5cot': [np.float64(0.00027602615418897806), np.float64(0.0011945087027758863), np.float64(0.00011220760179948583)], 'zerocot': [np.float64(9.041945186291192e-05), np.float64(0.0003912919863257885), np.float64(3.705074191511666e-05)], '3cot': [np.float64(0.0003044914448707172), np.float64(0.001317692817506926), np.float64(0.000123705718558539)], '1cot': [np.float64(0.00028346640527479437), np.float64(0.001226706538154778), np.float64(0.00011527685394990203)]}}
rows = []
for model, prompts in data.items():
    for prompt, values in prompts.items():
        rows.append({
            "model": model,
            "prompt": prompt,
            "energy": values[0],
            "wcf": values[1],
            "gwp": values[2],
        })

df = pd.DataFrame(rows)
df

,model,prompt,energy,wcf,gwp
0,gemini-2.5-pro,compact,0.027824,0.122789,0.010787
1,gemini-2.5-pro,few5,0.028831,0.127233,0.011149
2,gemini-2.5-pro,few3,0.027395,0.120893,0.010602
3,gemini-2.5-pro,few1,0.030040,0.132565,0.011628
4,gemini-2.5-pro,zero,0.028259,0.124705,0.010946
5,gemini-2.5-pro,5cot,0.029496,0.130166,0.011409
6,gemini-2.5-pro,zerocot,0.026393,0.116470,0.010217
7,gemini-2.5-pro,3cot,0.036282,0.160114,0.014040
8,gemini-2.5-pro,1cot,0.026414,0.116565,0.010219
9,gemini-2.5-flash,compact,0.001979,0.008732,0.000773


In [27]:
df.to_csv("updated_sus.csv",index=False)